# Generate tsunami plots and data #

## Set up Jupyter Notebook ##

In [ ]:
#set up environment and imported modules
import os
os.environ['CLAW'] = '/Users/emiddleton/Documents/python/clawpack/clawpack_src/clawpack'
os.environ['FC'] = 'gfortran'
dir = os.path.join(os.environ['CLAW'], 'geoclaw/examples/hokkaido')

%pylab inline

from clawpack.clawutil import nbtools
from clawpack.visclaw import animation_tools
from IPython.display import HTML, Image
from clawpack.geoclaw import dtopotools
import matplotlib.pyplot as plt
plt.rcParams["animation.embed_limit"] = 500 # 20 mb default is too small

import numpy as np

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [39]:
def show_anim(anim):
    html_version = HTML(anim.to_jshtml())
    # html_version = HTML(anim.to_html5_video())
    return html_version

## Compile and run Geoclaw Code

In [13]:
nbtools.make_exe(new=True, verbose=False)  # compile xgeoclaw

In [ ]:
# setrun data
# create *.data files from parameters in setrun.py
# parameters below are currently the same information as in the setrun file, but added here for easy access/changes
from setrun import setrun 
rundata = setrun()

# current refinement parameters
rundata.refinement_data.wave_tolerance = 0.02
rundata.amrdata.amr_levels_max = 3
rundata.amrdata.refinement_ratios_x = [6,8]
rundata.amrdata.refinement_ratios_y = [6,8]
rundata.amrdata.refinement_ratios_t = [6,8]

# regions and gauges
rundata.regiondata.regions = []
# to specify regions of refinement append lines of the form
#  [minlevel,maxlevel,t1,t2,x1,x2,y1,y2]
    
if 1:
    rundata.regiondata.regions.append([1, 1, 0., 1e9, -180,180,-90,90])
    rundata.regiondata.regions.append([1, 3, 0., 1e9, 141, 146, 41, 43])
    rundata.regiondata.regions.append([3, 3, 0., 1e9, 142, 144, 41.5, 42.5])
    rundata.regiondata.regions.append([3, 3, 0., 1e9, 144.5, 145.5, 42.5, 43.5])
    #rundata.regiondata.regions.append([3, 3, 0., 1e9, 142, 144, 41, 43])

# ---------------
# Gauges:
# ---------------
rundata.gaugedata.gauges = []
# # for gauges append lines of the form  [gaugeno, x, y, t1, t2]
rundata.gaugedata.gauges.append([129, 142.755637, 42.148995, 0., 1.e10]) # urakawa
rundata.gaugedata.gauges.append([111, 144.897712, 42.978294, 0., 1.e10]) # mabiro

# If you want to change output times, go to setrun.py, search section Output times:
# if you want to change computational domain go to setrun.py, search section Spatial domain:

rundata.write()      

# in the setrun file, I commented out the line about kml tools so hopefully it will stop generating those
# I also changed the topography file from .asc to the curr_topo.tt3 so hopefully that doesn't break anything 

In [41]:
run maketopo.py  # download the topo file and create the dtopo file

*** Note: since grid registration is llcorner,
    will shift x,y values by (dx/2, dy/2) to cell centers
The extent of the data in longitude and latitude: 
[np.float64(135.0020833333335), np.float64(151.9979166680265), np.float64(34.5020833333335), np.float64(46.9979166676665)]
Using Okada model to create dtopo file


## Output visuals for the dtopo created in maketopo.py

In [ ]:
# copied from maketopo.py and CSZ_example.ipynb

fault_geometry_file = os.path.join(dir, 'tok_coords.csv') # finite fault mesh
rupture_file = os.path.join(dir, 'tok_rupt_param.csv') # rupture information as [slip, rise, rupt time]

fault_mesh = np.loadtxt(fault_geometry_file, delimiter=",", skiprows=1) #path, comma separated values, first row is a header
fault_mesh[:,[2,5,8]] = 1e3*abs(fault_mesh[:,[2,5,8]]) #array slicing accesses depth element, changing it to be positive m instead of negative km
rupture_parameters = np.loadtxt(rupture_file, delimiter=",", skiprows=1)

# create fault object
fault0 = dtopotools.Fault()
fault0.subfaults = []
fault0.rupture_type = 'kinematic'

J = int(np.floor(fault_mesh.shape[0]))

# read in sub fault and rupture information
for j in range(J):
    subfault0 = dtopotools.SubFault()
    node1 = fault_mesh[j,0:3].tolist()
    node2 = fault_mesh[j,3:6].tolist()
    node3 = fault_mesh[j,6:9].tolist()
    node_list = [node1,node2,node3]

    subfault0.set_corners(node_list,projection_zone='10')
    subfault0.rupture_time = rupture_parameters[j,2]
    subfault0.rise_time = rupture_parameters[j,1]
    subfault0.rake = fault_mesh[j,9]

    subfault0.slip = rupture_parameters[j,0]
    fault0.subfaults.append(subfault0)

print("Using Okada model to create dtopo file")

x,y = fault0.create_dtopo_xy(dx = 4/60.) #what is dx?
print('Will create dtopo on arrays of shape %i by %i' % (len(x),len(y)))
tfinal = max([subfault1.rupture_time + subfault1.rise_time for subfault1 in fault0.subfaults])
times0 = np.linspace(0.,tfinal,100)
dtopo0 = fault0.create_dtopography(x,y,times=times0,verbose=False);
print('Created %s, with dynamic rupture of a Mw %.2f event' % ("tokachi-oki", fault0.Mw()))


# copied directly from CSZ_example, plots the fault

# fig = plt.figure(figsize=(15,10))
# #ax = fig.add_subplot(121, projection='3d')
# ax = fig.add_axes([.05,.05,.65,.9], projection='3d')
# for s in fault0.subfaults:
#     c = s.corners #list of 3 nodes
#     c.append(c[0]) #does this close a loop or something?
#     c = np.array(c)
#     ax.plot(c[:,0],c[:,1],-c[:,2]/1000.,color='b') #depth is negative and km (m/1000)
# ax.view_init(10,60)
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.set_zlabel('Depth (km)')
# ax.set_title('Triangular subfaults')

# #ax = fig.add_subplot(122)
# ax = fig.add_axes([.75,.05,.2,.9])
# for s in fault0.subfaults:
#     c = s.corners
#     c.append(c[0])
#     c = np.array(c)
#     ax.plot(c[:,0],c[:,1], 'b')
# ax.set_aspect(1./np.cos(45*np.pi/180.))
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.set_title('Plan view')

plt.close('all')

In [ ]:
# show uplift (copied from CSZ_example.ipynb)

plt.close('all')

fig,(ax0,ax1,ax2, ax3) = plt.subplots(ncols=4,nrows=1,figsize=(16,6))
fault0.plot_subfaults(axes=ax0,slip_color=True,plot_box=False);
ax0.set_title('Slip on Fault');

X = dtopo0.X; Y = dtopo0.Y; dZ_at_t = dtopo0.dZ_at_t
dz_max = dtopo0.dZ.max()

t0 = 0.25*tfinal    # time to plot deformation
dtopotools.plot_dZ_colors(X,Y,dZ_at_t(t0),axes=ax1, 
                          cmax_dZ = dz_max, add_colorbar=False);
ax1.set_title('Seafloor at time t=' + str(t0));

t0 = 0.5*tfinal    # time to plot deformation
dtopotools.plot_dZ_colors(X,Y,dZ_at_t(t0),axes=ax2,
                          cmax_dZ = dz_max, add_colorbar=False);

ax2.set_title('Seafloor at time t=' + str(t0));

t0 = tfinal    # time to plot deformation
dtopotools.plot_dZ_colors(X,Y,dZ_at_t(t0),axes=ax3,
                          cmax_dZ = dz_max, add_colorbar=True);
ax3.set_title('Seafloor at time t=' + str(t0));

In [ ]:
#Animate the rupture (from CSZ_example)

dz_max = abs(dtopo0.dZ).max()

# Incorporate this function in dtopotools to replace animate_dz_colors?
def plot_subfaults_dZ(t, fig):
    fig.clf()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    fault0.plot_subfaults(axes=ax1, slip_color=True, plot_box=False,
                          slip_time=t)
    dtopo0.plot_dZ_colors(axes=ax2, t=t, cmax_dZ=dz_max)
    return fig

figs = []
times = dtopo0.times[::5]   # only use every 5th time for animation
if dtopo0.times[-1] not in times:
    times = np.hstack((times, dtopo0.times[-1]))  # include final dz
    
print('Animation will include %i times' % len(times))
for k,t in enumerate(times):
    fig = plt.figure(figsize=(12,5))
    plot_subfaults_dZ(t,fig)
    figs.append(fig)
    plt.close(fig)

anim = animation_tools.animate_figs(figs, figsize=(12,6))
HTML(anim.to_jshtml())

## Generate tsunami animation and plots

In [ ]:
# function definitions for 'make' subprocesses
# function copied and edited from the one given in nbtools.py that wouldn't work right

def make_driver(args, env, outfile, verbose):

    """
    Use subprocess to run a make command and capture the output.

    :Input:
    - *args*: arguments to the make command
    - *env*: environment to run in (if None, use *os.environ*)
      Useful if $CLAW must be set in notebook.
    - *outfile*: file name of file for capturing stdout and stderr
    - *verbose*: if True, print out command and display link to output file
    
    If the return code is non-zero, print a warning and link to output file
    regardless of value of *verbose*.

    """

    import subprocess # execute terminal commands in python
    import os, sys

    if env is None:
        env = os.environ

    if verbose:
        print("Executing shell command:   make %s" % args)
        sys.stdout.flush()

    ofile = open(outfile,'w')
    cmd_list = ['make'] + args.split()

    # location of Makefile
    directory = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido' 
    cmd_list.insert(1, "-C")
    cmd_list.insert(2, directory)

    job = subprocess.Popen(cmd_list, stdout=ofile,stderr=ofile,env=env)
    return_code = job.wait()
    errors = (return_code != 0)
    if errors:
        print("*** Possible errors, return_code = %s" % return_code)
    
    if verbose or errors:
        local_file = FileLink(outfile)
        print("Done...  Check this file to see output:") 
        display(local_file)

def make_plots(label=None, env=None, verbose=True):
    """Perform 'make plots' and display links"""

    if label is None: 
        label = ''
    else:
        if label[0] != '_':
            label = '_' + label
    #outdir = '_output%s' % str(label)
    outdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_output'
    #plotdir = '_plots%s' % str(label)
    plotdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_plots'
    outfile = 'plot_output%s.txt' % str(label)

    args = '.plots OUTDIR=%s PLOTDIR=%s' % (outdir,plotdir)
    #args = '.plots'
    make_driver(args, env, outfile, verbose)

    if verbose:
        index_file = FileLink('%s/_PlotIndex.html' % plotdir)
        print("View plots created at this link:")
        display(index_file)

    return plotdir

In [ ]:
# run make .plots (LONG RUNTIME)

plotdir = make_plots(verbose=False)

In [ ]:
#plotdir = '/Users/anitamiddleton/Documents/python/clawpack/clawpack_src/clawpack/geoclaw/examples/hokkaido/_plots'
anim = animation_tools.animate_from_plotdir(plotdir, figno=0) #figno=1 shows cropped area around hokkaido coastline
show_anim(anim) 

### output gauge data

In [ ]:
Image('_plots/gauge0129fig300.png', width=400)

In [ ]:
Image('_plots/gauge0111fig300.png', width=400)

In [ ]:
# Gauge located near Urakawa 142.755637, 42.158995
from setplot import setplot
plotdata = setplot()
plotdata.outdir = '_output'
g129 = plotdata.getgauge(129)
t = g129.t / 3600.  # convert to hours
eta = g129.q[3,:]   # eta = h + B (depth plus bathymetry)
plot(t,eta)
xlim(0,4)
ylim(-3, 3)
xlabel('Hours since earthquake')
ylabel('meters')
title('Sea surface elevation at gauge 129')
grid(True)

In [ ]:
# Gauge located near Mabiro, 144.897712, 42.988294
from setplot import setplot
plotdata = setplot()
plotdata.outdir = '_output'
g111 = plotdata.getgauge(111)
t = g111.t / 3600.  # convert to hours
eta = g111.q[3,:]   # eta = h + B (depth plus bathymetry)
plot(t,eta)
xlim(0,4)
ylim(-1, 1)
xlabel('Hours since earthquake')
ylabel('meters')
title('Sea surface elevation at gauge 111')
grid(True)